# 地理空間データの可視化

## ポイントの可視化

In [1]:
import pandas as pd

itami_wifi = pd.read_csv("data/282073publicwirelesslan202204.csv")
itami_wifi.iloc[:3, 3:10]

,名称,名称_カナ,名称_英語,住所,方書,緯度,経度
0,有岡城跡公園,アリオカジョウセキコウエン,Arioka Castle Ruins,兵庫県伊丹市伊丹1-16,NaN,34.780851,135.421042
1,市バス総合案内所,シバスソウゴウアンナイジョ,City Bus Terminal,兵庫県伊丹市西台1-1-24,NaN,34.780171,135.413086
2,市立伊丹ミュージアム,シリツイタミミュージアム,Itami City Museum,兵庫県伊丹市宮ノ前2-5,NaN,34.781996,135.417263


In [2]:
itami_center = itami_wifi.loc[:, ["緯度", "経度"]].mean()
itami_center

緯度     34.781261
経度    135.422682
dtype: float64

In [3]:
import folium

itami_wifi_map = folium.Map(
    location=itami_center,
    zoom_start=15.2,
    width=800,
    height=400,
)
itami_wifi_map

ModuleNotFoundError: No module named 'folium'

In [4]:
marker = folium.Marker(location=itami_wifi.loc[0, ["緯度", "経度"]])
marker.add_to(itami_wifi_map)
itami_wifi_map

NameError: name 'folium' is not defined

In [5]:
itami_wifi.loc[:, ["緯度", "経度"]].apply(
    lambda x: folium.Marker(location=x).add_to(itami_wifi_map),
    axis=1,
)
itami_wifi_map

NameError: name 'folium' is not defined

In [6]:
import geopandas as gpd

land_price_gdf = (
    gpd.read_file(
        "https://nlftp.mlit.go.jp/ksj/gml/data/L01/L01-22/L01-22_25_GML.zip"
    )
    .groupby("L01_005")
    .get_group("2022")
)
land_price_gdf.loc[:3, ["L01_006", "L01_023", "geometry"]]

ModuleNotFoundError: No module named 'geopandas'

In [7]:
land_price = land_price_gdf.apply(
    lambda s: pd.Series(
        (
            s["geometry"].y,
            s["geometry"].x,
            s["L01_006"],
        )
    ),
    axis=1,
).values
land_price[:3]

NameError: name 'land_price_gdf' is not defined

In [8]:
from folium.plugins import HeatMap

shiga_center = (
    land_price_gdf.loc[:, "geometry"].y.mean(),
    land_price_gdf.loc[:, "geometry"].x.mean(),
)
land_price_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
HeatMap(
    data=land_price,
    radius=12,
).add_to(land_price_map)
land_price_map

ModuleNotFoundError: No module named 'folium'

## ラインの可視化

In [9]:
minx, miny = land_price_gdf.loc[:, "geometry"].bounds.min()[
    ["minx", "miny"]
]
maxx, maxy = land_price_gdf.loc[:, "geometry"].bounds.max()[
    ["maxx", "maxy"]
]
rail_gdf = gpd.read_file(
    "https://nlftp.mlit.go.jp/ksj/gml/data/N02/N02-21/N02-21_GML.zip"
).cx[minx:maxx, miny:maxy]
rail_gdf.head()

NameError: name 'land_price_gdf' is not defined

In [10]:
# 京津線のデータを抽出
keishinsen = rail_gdf.groupby("N02_003")["geometry"].get_group("京津線")
# 中心となる座標
keishinsen_center = (
    keishinsen.map(lambda p: p.centroid.y).mean(),
    keishinsen.map(lambda p: p.centroid.x).mean(),
)
keishinsen_center

NameError: name 'rail_gdf' is not defined

In [11]:
import numpy as np


keishinsen_map = folium.Map(
    location=keishinsen_center,
    zoom_start=14,
)
keishinsen.apply(
    lambda s: folium.PolyLine(
        # 緯度, 経度の順番に変換、fliplr関数で順序を反転
        locations=np.fliplr(s.coords),
        weight=15  # 線の太さ
    ).add_to(keishinsen_map)
)
keishinsen_map

NameError: name 'folium' is not defined

In [12]:
rail_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.features.GeoJson(rail_gdf).add_to(rail_map)
rail_map

NameError: name 'folium' is not defined

## ポリゴンの可視化

In [13]:
did_gdf = gpd.read_file("./data/A16-15_25_DID.geojson")
did_gdf.head()

NameError: name 'gpd' is not defined

In [14]:
did_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.GeoJson(did_gdf).add_to(did_map)
did_map

NameError: name 'folium' is not defined

In [15]:
did_choroplethmap = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.features.Choropleth(
    geo_data=did_gdf,
    data=did_gdf,
    key_on="feature.properties.DIDid",
    columns=["DIDid", "人口"],
    legend_name="人口",
).add_to(did_choroplethmap)
did_choroplethmap

NameError: name 'folium' is not defined